# Feature Selecection

In [45]:
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message=".*use_label_encoder.*")

# General
import numpy as np

# Data Management
import pandas as pd
from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

# Import Preporcessed Data

In [46]:
df = pd.read_csv("data/BTC-USD.csv")
df.set_index("Date" , inplace=True)

# Add prediction target

In [47]:
# Specify Target
df.loc[df["Range"].shift(-1) > df["AVG_Range"] , "TARGET"] = 1
df.loc[df["Range"].shift(-1) <= df["AVG_Range"] , "TARGET"] = 0

In [48]:
# Check for NAN
nan_location = np.where(np.isnan(df))

In [49]:
# Fill NaNs
df["TARGET"].fillna(0,inplace=True)
df.tail()

,Open,High,Low,Close,Volume,Returns,Range,Bench_C_Rets,RSI,RSI_Return,...,DOW,Rolling_rets,AVG_Range,Returns_T1,Range_T1,Rolling_rets_T1,Returns_T2,Range_T2,Rolling_rets_T2,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-23,0.003965,0.014381,0.010036,27225.726562,0.238807,0.013945,0.023065,3.144802,46.178235,1.112756,...,1,-0.009218,0.032783,0.003643,0.018682,-0.031291,-0.013851,0.020931,-0.015115,1.0
2023-05-24,0.013727,-0.007657,-0.026462,26334.818359,0.189959,-0.032723,0.042826,3.008356,38.323938,0.829913,...,2,-0.039547,0.033091,0.013945,0.023065,-0.009218,0.003643,0.018682,-0.031291,0.0
2023-05-25,-0.032880,-0.023254,-0.008273,26476.207031,-0.150191,0.005369,0.027073,3.029568,40.066179,1.045461,...,3,-0.062598,0.032569,-0.032723,0.042826,-0.039547,0.013945,0.023065,-0.009218,0.0
2023-05-26,0.005497,0.012228,0.017510,26719.291016,-0.082268,0.009181,0.021740,3.066700,43.045036,1.074348,...,4,-0.057483,0.030035,0.005369,0.027073,-0.062598,-0.032723,0.042826,-0.039547,0.0
2023-05-27,0.009292,-0.001032,0.010522,26868.353516,-0.379150,0.005579,0.010058,3.089815,44.855043,1.042049,...,5,-0.088884,0.028646,0.009181,0.021740,-0.057483,0.005369,0.027073,-0.062598,0.0


# Train - Test SPLIT

In [50]:
# REMOVE UNWANTED COLUMNS (dont put information about the future in your data other than the target)
df_tts = df.copy()
df_tts.drop(columns=["Close", "Bench_C_Rets" , "Open", "High", "Low"], inplace=True)

In [51]:
# Split into Learning (X) and Target (y) data
X = df_tts.iloc[: , : -1 ]
y = df_tts.iloc[: , -1 ]

In [52]:
# Train Test Split
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size=0.2 , random_state=42)
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

Shape of X_train: (968, 16)
Shape of y_train: (968,)


# Build Initial Model
- find the useful features

In [53]:
# Select type of model to optimize for
is_binary = True
is_optimize_for_precision = True

In [54]:
# Determine Objective and Eval Metrics
if is_binary:
    objective = "binary:logistic"
    eval_metric = "logloss"
    eval_metric_list = ["error","logloss",eval_metric]
else:
    objective = "multi:softmax"
    eval_metric = "mlogloss"
    eval_metric_list = ["merror","mlogloss",eval_metric]

In [55]:
# Refine Eval Metric
if is_binary and is_optimize_for_precision:
    eval_metric = "aucpr"
    scoring = "precision"
elif is_binary and not is_optimize_for_precision:
    eval_metric = "auc"
    scoring = "f1"
else:
    scoring = "accuracy"

In [56]:
# Build First Classifier Model
classifier_0 = XGBClassifier(
    objective=objective,
    booster="gbtree",
    eval_metric=eval_metric,
    subsample = 0.8,
    colsample_bytree=1,
    random_state=1,
    use_label_encoder=False
)

d:\Programs\Anaconda\envs\ML-finance\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


# Find Hyper-Parameters
- find optimal
    - estimators 
    - learning rate 
    - maximum depth
    - gamma

In [57]:
# Provide Grid for Hyper-Parameterss
param_grid = {
    "gamma": [0, 0.1, 0.2, 0.5, 1, 1.5, 2, 3, 12, 20],
    "learning_rate": [0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8],
    "max_depth": [1, 2, 3, 4, 5, 6, 8, 12],
    "n_estimators": [25, 50, 65, 80, 100, 115, 200]
}

In [58]:
# preform Random Search for Best Hyper Params
grid_search = RandomizedSearchCV(estimator=classifier_0, param_distributions=param_grid, scoring=scoring)
best_model = grid_search.fit(X_train,y_train)
hyperparams = best_model.best_params_
ne = hyperparams["n_estimators"]
lr = hyperparams["learning_rate"]
md = hyperparams["max_depth"]
gm = hyperparams["gamma"]


print("Recommended params >>" , f"ne: {ne}", f"lr: {lr}", f"md: {md}", f"gm: {gm}")

d:\Programs\Anaconda\envs\ML-finance\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
d:\Programs\Anaconda\envs\ML-finance\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
d:\Programs\Anaconda\envs\ML-finance\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
d:\Programs\Anaconda\envs\ML-finance\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
d:\Programs\Anaconda\envs\ML-finance\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Recommended params >> ne: 25 lr: 0.01 md: 2 gm: 1


d:\Programs\Anaconda\envs\ML-finance\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
